In [1]:
import numpy as np
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import backend as K
K.set_image_dim_ordering('th')
import os
os.chdir("/home/ubuntu/data/")
import time
import matplotlib.pyplot as plt

from IPython.display import Image, display, clear_output
import datetime

Using TensorFlow backend.


In [2]:
# path to the model weights files.
weights_path = '../vgg16_weights_tf_dim_ordering_tf_kernels.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'train'
validation_data_dir = 'validation'
test_dir = 'test'
nb_train_samples = 800
nb_validation_samples = 370
epochs = 2
batch_size = 5

input_tensor = Input(shape=(3,img_width,img_height))
base_model = applications.VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
#top_model.load_weights(top_model_weights_path)
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/keras/applications/vgg16.py:182: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


In [3]:
model.load_weights('last_and_finetuned_15fixed.h5')

In [4]:
files_benign=os.listdir(test_dir+"/Benign/")
files_malignant=os.listdir(test_dir+"/Malignant/")

In [5]:
def predict_image_class(file):
    #load the VGG16 model
    #model = applications.VGG16(include_top=False, weights='imagenet')
    #load the picture
    x = image.load_img(file, target_size=(img_width,img_height))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    print(x.shape)
    # process the picture through the first network this will be used as input
    #for the top model
    #array = model.predict(x)
    #build the top model
    #model = Sequential()
    #model.add(Flatten(input_shape=array.shape[1:]))
    #model.add(Dense(256, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(1, activation='sigmoid'))
    # and use the weigths trained before
    #model.load_weights(top_model_weights_path)
    class_predicted = model.predict(x)
    print(class_predicted)
    if class_predicted==1:
        print("malign") # dog
    else:
        print("benign") # cat
        
from keras.preprocessing import image

print(test_dir + "/Benign/" + files_benign[0])
predict_image_class(test_dir + "/Benign/" + files_benign[0])

test/Benign/ISIC_0010371.jpg
(1, 3, 150, 150)
[[ 0.64314294]]
benign


In [6]:
predict_image_class("validation/Benign/ISIC_0009874.jpg")
#"IMG_0845c1.JPG")

(1, 3, 150, 150)
[[ 0.]]
benign
